# TODO finish

In [1]:
import pandas as pd
import csv
import collections
import re
import pickle
import numpy as np

In [2]:
fileNeop = "../data/RTRT_NEOPLASI_corr.csv"
fileIsto = "../data/ISTOLOGIE_corr.csv" 
fileIcdo = "icdo-3-dict-en.p"

fileSaveCorpus = "corpusTemporalV3/corpusTemporal.p"
fileSaveValues = "corpusTemporalV3/valuesTemporalSede1.p"
fileSaveValuesS = "corpusTemporalV3/valuesTemporalSede2.p"
fileSaveValuesM = "corpusTemporalV3/valuesTemporalMorfo1.p"
fileSaveValuesB = "corpusTemporalV3/valuesTemporalMorfo2.p"
fileSaveTrainWordVectors = "corpusTemporalV3/trainWordVectors.txt"

In [3]:
encoding = 'iso-8859-1'

In [4]:
icdo = pickle.load(open(fileIcdo, 'rb'))

# Load neop

In [5]:
dfNeop = pd.read_csv(fileNeop, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False,
                     usecols = ['id_neopl', 'sede_icdo3',
                                'morfologia_icdo3', 'inserimento_data_neoplasi'],
                     dtype={'id_neopl':pd.np.float64, 'sede_icdo3':pd.np.str, 
                            'morfologia_icdo3':pd.np.str, 'inserimento_data_neoplasi':pd.np.str},
                    #parse_dates=['inserimento_data_neoplasi'], 
                    #date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
                    )

In [6]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [7]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [8]:
dfNeop = dfNeop.dropna('index')

In [9]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [10]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [11]:
dfNeop['inserimento_data_neoplasi'] =  pd.to_datetime(dfNeop['inserimento_data_neoplasi'], format='%Y-%m-%d %H:%M:%S')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [13]:
dfNeop.dtypes

id_neopl                            float64
inserimento_data_neoplasi    datetime64[ns]
sede_icdo3                           object
morfologia_icdo3                     object
dtype: object

In [14]:
dfNeop = dfNeop.sort_values(by=['inserimento_data_neoplasi'])

In [15]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
107276,108110.0,1990-04-20 00:00:00,C443,80003
78576,79172.0,1990-04-20 00:00:00,C341,80003
78595,79191.0,1990-04-24 00:00:00,C329,80703
78602,79198.0,1990-04-24 00:00:00,C509,85203
78601,79197.0,1990-04-24 00:00:00,C710,94003
...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001
295873,346866.0,2014-03-10 12:26:26,C169,81403


remove invalid sites

In [17]:
dfNeop['correctSede'] = dfNeop.sede_icdo3.map(lambda s: 
                                              (len(s)==4) and 
                                              (s[0] == "C") and 
                                              (int(s[1:3]) in icdo.keys()) and
                                              s[3] in [str(i) for i in range(10)]
                                             )
dfNeop = dfNeop[dfNeop.correctSede]

# TODO finish

In [ ]:
dfNeop['correctMorfo'] = dfNeop.sede_icdo3.map(lambda s: 
                                               len(s)==5 and 
                                               int(s[:4]) in icdo.keys() and
                                               int(s[4]) in [0,1,2,3,6,9]
                                              )
dfNeop = dfNeop[dfNeop.correctSede]

In [18]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede
107276,108110.0,1990-04-20 00:00:00,C443,80003,True
78576,79172.0,1990-04-20 00:00:00,C341,80003,True
78595,79191.0,1990-04-24 00:00:00,C329,80703,True
78602,79198.0,1990-04-24 00:00:00,C509,85203,True
78601,79197.0,1990-04-24 00:00:00,C710,94003,True
...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True
309849,346856.0,2014-02-14 11:53:40,C424,97651,True
309850,346857.0,2014-02-14 12:00:33,C619,80001,True
295873,346866.0,2014-03-10 12:26:26,C169,81403,True


In [19]:
dfNeop['y'] = dfNeop.sede_icdo3.map(lambda s: int(s[1:3]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dfNeop['yS'] = dfNeop.sede_icdo3.map(lambda s: int(s[3]))

ValueError: invalid literal for int() with base 10: 'C'

In [ ]:
dfNeop['yM'] = dfNeop.morfologia_icdo3.map(lambda m: int(m[:4]))

In [ ]:
dfNeop['yB'] = dfNeop.morfologia_icdo3.map(lambda m: int(m[4]))

In [ ]:
dfNeop

# Load isto

In [ ]:
dfIsto = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [ ]:
dfIsto

In [ ]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfIsto['notizie'] = dfIsto.notizie.map(nullToEmpty)
dfIsto['diagnosi'] = dfIsto.diagnosi.map(nullToEmpty)
dfIsto['macroscopia'] = dfIsto.macroscopia.map(nullToEmpty)

In [ ]:
dfIsto = dfIsto.dropna('index')

In [ ]:
dfIsto

In [ ]:
dfIsto['text'] = dfIsto[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

In [ ]:
dfIsto['text'] = dfIsto['text'].astype('unicode')

In [ ]:
dfIsto['text'] = dfIsto['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

In [ ]:
dfIsto

In [ ]:
dfIsto['correctText'] = dfIsto.text.map(lambda t: len(t) > 0)

In [ ]:
dfIsto = dfIsto[dfIsto.correctText]

In [ ]:
dfIsto

In [ ]:
dfIsto = dfIsto.drop_duplicates(['text'])

In [ ]:
dfIsto

In [ ]:
dfIsto = dfIsto.drop_duplicates(['id_neopl'], keep=False)

In [ ]:
dfIsto

# Merge

In [ ]:
dfMerge = pd.merge(dfIsto, dfNeop, on='id_neopl', validate="one_to_one")

In [ ]:
dfMerge

# Unlabelled

In [ ]:
dfUn = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [ ]:
dfUn

In [ ]:
dfUn = dfUn[~dfUn.id_fonte.isin(dfMerge.id_fonte)]

In [ ]:
dfUn

In [ ]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfUn['notizie'] = dfUn.notizie.map(nullToEmpty)
dfUn['diagnosi'] = dfUn.diagnosi.map(nullToEmpty)
dfUn['macroscopia'] = dfUn.macroscopia.map(nullToEmpty)

In [ ]:
dfUn['text'] = dfUn[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

In [ ]:
dfUn['text'] = dfUn['text'].astype('unicode')

In [ ]:
dfUn['text'] = dfUn['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

In [ ]:
dfUn['correctText'] = dfUn.text.map(lambda t: len(t) > 0)

In [ ]:
dfUn = dfUn[dfUn.correctText]

In [ ]:
dfUn

In [ ]:
dfUn = dfUn[~dfUn.text.isin(dfMerge.text)]

In [ ]:
dfUn

# Final

In [ ]:
dfFinal = dfMerge[['inserimento_data_neoplasi','text', 'y', 'yS', 'yM', 'yB']]

In [ ]:
dfFinal = dfFinal.rename(columns={"inserimento_data_neoplasi": "data"})

In [ ]:
dfFinal = dfFinal.sort_values(by=['data'])

In [ ]:
dfFinal

In [ ]:
dfUnlabelled = dfUn[['text']]

In [ ]:
dfUnlabelled

In [ ]:
c = collections.Counter(dfFinal.y)
c.most_common()

In [ ]:
testLen = int(len(dfFinal)/100*20)

In [ ]:
dfTest = dfFinal.tail(testLen)

In [ ]:
dfTrain = dfFinal.head(len(dfFinal) - testLen)

In [ ]:
dfValid = dfTrain.tail(testLen)

In [ ]:
dfTrain = dfTrain.head(len(dfTrain) - testLen)

In [ ]:
dfTrain

In [ ]:
dfValid

In [ ]:
dfTest

In [ ]:
cTrain = collections.Counter(dfTrain.y)
cTrain.most_common()

In [ ]:
cValid = collections.Counter(dfValid.y)
cValid.most_common()

In [ ]:
cTest = collections.Counter(dfTest.y)
cTest.most_common()

In [ ]:
cTest[44]

In [ ]:
toRemove = set()
for i in cTrain.keys():
    if not i in cTest.keys():
        toRemove.add(i)
    

In [ ]:
toRemove

In [ ]:
for i in cValid.keys():
    if not i in cTest.keys() or not i in cTrain.keys():
        toRemove.add(i)

In [ ]:
toRemove

In [ ]:
for i in cTest.keys():
    if not i in cTrain.keys():
        toRemove.add(i)

In [ ]:
toRemove

In [ ]:
dfTrain = dfTrain[~dfTrain.y.isin(toRemove)]

In [ ]:
dfValid = dfValid[~dfValid.y.isin(toRemove)]

In [ ]:
dfTest = dfTest[~dfTest.y.isin(toRemove)]

In [ ]:
corpus = {
    'train':{
        'text':dfTrain['text'].values,
        'sede1':dfTrain['y'].values,
        'sede2':dfTrain['yS'].values,
        'morfo1':dfTrain['yM'].values,
        'morfo2':dfTrain['yB'].values,
    },
    'valid':{
        'text':dfValid['text'].values,
        'sede1':dfValid['y'].values,
        'sede2':dfValid['yS'].values,
        'morfo1':dfValid['yM'].values,
        'morfo2':dfValid['yB'].values,
    },
    'test':{
        'text':dfTest['text'].values,
        'sede1':dfTest['y'].values,
        'sede2':dfTest['yS'].values,
        'morfo1':dfTest['yM'].values,
        'morfo2':dfTest['yB'].values,
    },
}

In [ ]:
count = collections.Counter(np.concatenate((corpus['train']['sede1'], corpus['valid']['sede1'], corpus['test']['sede1'])))

In [ ]:
values = sorted(count.keys())

In [ ]:
countS = collections.Counter(np.concatenate((corpus['train']['sede2'], corpus['valid']['sede2'], corpus['test']['sede2'])))
valuesS = sorted(countS.keys())

In [ ]:
countM = collections.Counter(np.concatenate((corpus['train']['morfo1'], corpus['valid']['morfo1'], corpus['test']['morfo1'])))
valuesM = sorted(countM.keys())

In [ ]:
countB = collections.Counter(np.concatenate((corpus['train']['morfo2'], corpus['valid']['morfo2'], corpus['test']['morfo2'])))
valuesB = sorted(countB.keys())

In [ ]:
wordVectorsText = np.concatenate((dfUnlabelled.text.values, dfTrain.text.values, dfValid.text.values))

In [ ]:
wordVectorsText[:10]

In [ ]:
np.random.shuffle(wordVectorsText)

In [ ]:
wordVectorsText[:10]

# write

In [ ]:
pickle.dump(corpus, open(fileSaveCorpus, 'wb'))
pickle.dump(values, open(fileSaveValues, 'wb'))
pickle.dump(valuesS, open(fileSaveValuesS, 'wb'))
pickle.dump(valuesM, open(fileSaveValuesM, 'wb'))
pickle.dump(valuesB, open(fileSaveValuesB, 'wb'))
np.savetxt(fileSaveTrainWordVectors, wordVectorsText, delimiter=" ", fmt="%s")